<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/korean_QA_from_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a QA System with BERT on Wikipedia

https://qa.fastforwardlabs.com/pytorch/hugging%20face/wikipedia/bert/transformers/2020/05/19/Getting_Started_with_QA.html

위의 내용을 한국어 QA로 변경

In [1]:
!pip install transformers==3
!pip install wikipedia

     |████████████████████████████████| 757kB 5.1MB/s 
     |████████████████████████████████| 3.0MB 8.0MB/s 
     |████████████████████████████████| 890kB 28.0MB/s 
     |████████████████████████████████| 1.1MB 34.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=d274f02b27e13d2eefddc66142d48ea287f862cceaa0785a94c0dd0ab27be65b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=95b5e6c7bcd37409236bdcb0c513e2589aeefee28ce737e7520aac6b5b494272
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# executing these commands for the first time initiates a download of the 
# model weights to ~/.cache/torch/transformers/
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad") 
model = AutoModelForQuestionAnswering.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")

Some weights of the model checkpoint at monologg/koelectra-base-v3-finetuned-korquad were not used when initializing ElectraForQuestionAnswering: ['electra.embeddings.position_ids']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
question = "마케도니아를 통치 한 왕조는?"

context = """마케도니아는 고대 그리스와 고전 그리스 주변의 고대 왕국이었습니다.
그리고 나중에 헬레니즘 그리스의 지배 국가. 왕국이 설립되고 처음에 통치
Argead 왕조, Antipatrid 및 Antigonid 왕조가 그 뒤를이었습니다. 고대의 고향
마케도니아인, 그리스 반도의 북동부에서 시작되었습니다. 4 일 이전
기원전 세기, 아테네의 도시 국가가 지배하는 지역 밖의 작은 왕국이었습니다.
스파르타와 테베, 그리고 잠시 아케 메니 드 페르시아에 종속됩니다."""


# 1. TOKENIZE THE INPUT
# note: if you don't include return_tensors='pt' you'll get a list of lists which is easier for 
# exploration but you cannot feed that into a model. 
inputs = tokenizer.encode_plus(question, context, return_tensors="pt") 

# 2. OBTAIN MODEL SCORES
# the AutoModelForQuestionAnswering class includes a span predictor on top of the model. 
# the model returns answer start and end scores for each word in the text
answer_start_scores, answer_end_scores = model(**inputs)

answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score

# 3. GET THE ANSWER SPAN
# once we have the most likely start and end tokens, we grab all the tokens between them
# and convert tokens back to words!
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

'Argead 왕조'

In [8]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import wikipedia as wiki
import pprint as pp
from collections import OrderedDict

class DocumentReader:
    def __init__(self, pretrained_model_name_or_path='monologg/koelectra-base-v3-finetuned-korquad'):
        self.READER_PATH = pretrained_model_name_or_path
        self.tokenizer = AutoTokenizer.from_pretrained(self.READER_PATH)
        self.model = AutoModelForQuestionAnswering.from_pretrained(self.READER_PATH)
        self.max_len = self.model.config.max_position_embeddings
        self.chunked = False

    def tokenize(self, question, text):
        self.inputs = self.tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
        self.input_ids = self.inputs["input_ids"].tolist()[0]

        if len(self.input_ids) > self.max_len:
            self.inputs = self.chunkify()
            self.chunked = True

    def chunkify(self):
        """ 
        Break up a long article into chunks that fit within the max token
        requirement for that Transformer model. 

        Calls to BERT / RoBERTa / ALBERT require the following format:
        [CLS] question tokens [SEP] context tokens [SEP].
        """

        # create question mask based on token_type_ids
        # value is 0 for question tokens, 1 for context tokens
        qmask = self.inputs['token_type_ids'].lt(1)
        qt = torch.masked_select(self.inputs['input_ids'], qmask)
        chunk_size = self.max_len - qt.size()[0] - 1 # the "-1" accounts for
        # having to add an ending [SEP] token to the end

        # create a dict of dicts; each sub-dict mimics the structure of pre-chunked model input
        chunked_input = OrderedDict()
        for k,v in self.inputs.items():
            q = torch.masked_select(v, qmask)
            c = torch.masked_select(v, ~qmask)
            chunks = torch.split(c, chunk_size)
            
            for i, chunk in enumerate(chunks):
                if i not in chunked_input:
                    chunked_input[i] = {}

                thing = torch.cat((q, chunk))
                if i != len(chunks)-1:
                    if k == 'input_ids':
                        thing = torch.cat((thing, torch.tensor([102])))
                    else:
                        thing = torch.cat((thing, torch.tensor([1])))

                chunked_input[i][k] = torch.unsqueeze(thing, dim=0)
        return chunked_input

    def get_answer(self):
        if self.chunked:
            answer = ''
            for k, chunk in self.inputs.items():
                answer_start_scores, answer_end_scores = self.model(**chunk)

                answer_start = torch.argmax(answer_start_scores)
                answer_end = torch.argmax(answer_end_scores) + 1

                ans = self.convert_ids_to_string(chunk['input_ids'][0][answer_start:answer_end])
                if ans != '[CLS]':
                    answer += ans + " / "
            return answer
        else:
            answer_start_scores, answer_end_scores = self.model(**self.inputs)

            answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
            answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
        
            return self.convert_ids_to_string(self.inputs['input_ids'][0][
                                              answer_start:answer_end])

    def convert_ids_to_string(self, input_ids):
        return self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(input_ids))

In [9]:
questions = [
    '물의 원자 구성은 어떻게 되는가?'
]

reader = DocumentReader("monologg/koelectra-base-v3-finetuned-korquad") 

# if you trained your own model using the training cell earlier, you can access it with this:
#reader = DocumentReader("./models/bert/bbu_squad2")
wiki.set_lang('ko')
for question in questions:
    print(f"Question: {question}")
    results = wiki.search(question)
    print(results)
    page = ""
    text = ""
    for result in results:
      try:
        page = wiki.page(result)
        print(f"Top wiki result: {page}")
        text = page.content
        reader.tokenize(question, text)
        print(f"Answer: {reader.get_answer()}")
      except Exception as ex:
        print(ex)


Some weights of the model checkpoint at monologg/koelectra-base-v3-finetuned-korquad were not used when initializing ElectraForQuestionAnswering: ['electra.embeddings.position_ids']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: 물의 원자 구성은 어떻게 되는가?
['원자', '물', '원자 번호', '화학', '원자론', '원자 반지름', '질소', '분자', '원자력 발전소', '화학식']
Top wiki result: <WikipediaPage '원자'>
Answer: 원소 / 기본적인 질량단위 / ° /  /  /  /  / 대략 1 . 07 A 3 { \ displaystyle 1 . 07 { \ sqrt [ { 3 } ] { A } } } 펨토미터로 근사된다 . 핵자들은 굉장히 짧은 거리에 작용하는 힘인 핵력으로 서로 결합되어있는데 , 2 . 5 펨토미터까지는 이 힘이 정전기력보다 강하기 때문에 양성자들이 서로 밀어내는 힘보다 끌어당기는 힘이 더 ° / ° / 원자로 / 
Top wiki result: <WikipediaPage '물'>
Answer: 산소와 수소 /  / [SEP] /  /  / [SEP]는 성질이 있다 . 철의 경우 반응성이 크나 직접적으로는 산소와 잘 반응하지 않으며 아주 천천히 산화철을 생성한다 . 하지만 물이 묻은 철은 상황이 다른데 , 그 원인은 물이 철을 이온화하면서 전자를 내놓고 이 전자를 받은 산소원자가 양이온으로 하전된 철 분자와 결합을 하면서 이루어지기 때문이다 . 이러한 산화는 물기가 완전히 없어질 때까지 멈추지 않아 결국 속까지 모두 산화시키고 만다 . 금속의 산화를 막기 위해 기름칠을 하는 경우가 많은데 이는 기름과 물 사이의 반발력을 이용한 것이다 . 한편 찬물에서 급격히 반응하는 금속은 포타슘 , 칼슘 , 소듐 등이 있고 , 뜨거운 물에서 급격히 반응하는 금속은 마그네슘 , 알루미늄 , 아연 등이 있다 . = = = 공학에서의 물 = = = 수리학에서 1기압 하에서 물의 단위 중량은 보통 [UNK] [UNK] 쓰며 , 1000kg중 / m³ ( 1t중 / m³ = 1g중 / cm³ ) 으로 나타낸다 . 지구상에서 물의 단위중량을 다룰 때는 편의상 ' 중 ' ( force ) 을 빼고 1000kg / m³ ( 1t 

여기서 부터는 최적의 답을 찾기 위해 문서 유사도 혼합

In [10]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 3.1MB/s 
     |████████████████████████████████| 1.3MB 5.6MB/s 
     |████████████████████████████████| 2.9MB 14.9MB/s 
     |████████████████████████████████| 1.1MB 32.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101035 sha256=04c0df7702be15af734c9ec7dc684778b79e27df20f749c017d4806115653d4f
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
Successfully built sentence-transformers
  Found existing installation: tokenizers 0.8.0rc4
    Uninstalling tokenizers-0.8.0rc4:
      Successfully uninstalled tokenizers-0.8.0rc4
  Found existing installation: sentencepiece 0.1.94
    Uninstalling sentencepiece-0.1.94:
      Successfully uninstalled sentencepiece-0.1.94
  Found existing installation: transformers 3.0.0
    Uninstalling transformers-3.0.0:
      Successfully uninstalled transformers-3.0.0


In [11]:
from sentence_transformers import SentenceTransformer
# embedder download...
embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

100%|██████████| 1.80G/1.80G [01:56<00:00, 15.5MB/s]


In [ ]:

questions = [
    '물의 원자 구성은 어떻게 되는가?'
]

reader = DocumentReader("monologg/koelectra-base-v3-finetuned-korquad") 

# if you trained your own model using the training cell earlier, you can access it with this:
#reader = DocumentReader("./models/bert/bbu_squad2")
wiki.set_lang('ko')
corpus = []
for question in questions:
    print(f"Question: {question}")
    results = wiki.search(question)
    print(results)
    page = ""
    text = ""
    for result in results:
      try:
        page = wiki.page(result)
        print(f"Top wiki result: {page}")
        text = page.content
        corpus.add(text)
        reader.tokenize(question, text)

      except Exception as ex:
        print(ex)

corpus_embeddings = embedder.encode(corpus,show_progress_bar=True) 